## Генерация параллельных предложений 2.0

В первой тетрадке generate_out_of_existent.ipynb мы заменяли только наречия, а в этой будем заменять:

* существительные в именительном падеже
* существительные в винительном падеже
* переходные глаголы
* (непереходные тоже)

In [1]:
#!g1.1
import os, sys
import re, random
import copy
import json
import pympi
import pymystem3
import pandas as pd
import numpy as np
from pprint import pprint
from itertools import chain
from tqdm import tqdm

sys.path.append('../')
from pyeaf.pyeaf import EAFReader
from pyeaf.text import VocabularyVectorizer, TextStemmer, RSLStemmer, GramBinarizer

In [5]:
!unzip Разметки_eaf-20211027.zip

Archive:  Разметки_eaf-20211027.zip
  inflating: Разметки eaf/Паленный_13_КАК.eaf  
  inflating: Разметки eaf/Салий_2_ЕСЯ.eaf  
  inflating: Разметки eaf/Шец1_ВПР.eaf  
  inflating: Разметки eaf/Добрые_истории_5_ААС.eaf  
  inflating: Разметки eaf/бальзам_ААС.eaf  
  inflating: Разметки eaf/диктант5_СГБ.eaf  
  inflating: Разметки eaf/Бедная_девочка_СГБ.eaf  
  inflating: Разметки eaf/публикации_АМК.eaf  
  inflating: Разметки eaf/Сокол_5_ААС.eaf  
  inflating: Разметки eaf/соловей_и_роза_часть_4_СГБ.eaf  
  inflating: Разметки eaf/Золушка_сцена_5_ВПР.eaf  
  inflating: Разметки eaf/Каша_из_топора_ВПР.eaf  
  inflating: Разметки eaf/Золушка_сцена_6_ВПР.eaf  
  inflating: Разметки eaf/соловей_и_роза_часть_5_СГБ.eaf  
  inflating: Разметки eaf/диктант3_СГБ.eaf  
  inflating: Разметки eaf/времена_года_ККД.eaf  
  inflating: Разметки eaf/диктант1_СГБ.eaf  
  inflating: Разметки eaf/диктант2_СГБ.eaf  
  inflating: Разметки eaf/Лев_и_мышь_ВПР.eaf  
  inflating: Разметки eaf/зарплату_маме_СГБ

In [4]:
#!g1.1
er = EAFReader(directory="Разметки eaf")
er.load()

## Stem
Читаем предложения и делаем им стемминг и грамматический анализ

In [5]:
#!g1.1
sentences_rus = er.get_sentences(er.RUS)
sentences_rsl = er.get_sentences(er.RSL_R)

# sentences_rsl_left = er.get_sentences(er.RSL_L)

In [6]:
#!g1.1
sentences_rus[:5]

['Видно меня?',
 'Там было ограничение по сумме, и ВОГ начал перевозить товары в Россию',
 'Сам ВОГ не мог перевозить, поэтому этим управляли крутые слышащие, которым платили проценты',
 'Начали крышевать ВОГ',
 'Абрамов приехал в Москву с просьбой']

In [5]:
#!g1.1
#%pip install pymystem3

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [7]:
#!g1.1
st = TextStemmer()

In [8]:
#!g1.1

stem_sentences_rus, gram_sentences_rus = st.stem(sentences_rus, gram=True)
stem_sentences_rsl = list(map(RSLStemmer.stem_sentence, sentences_rsl))

Так происходит потому что бывают левши, а еще бывают короткие предложения которые показывают одной ркуой левой.

Типа "следующий слайд" можно левой рукой показать махнуть, даже если ты правша

Это надо потом учесть иначе около 240 предложений выпадут из генерации

In [10]:
#!g1.1
# Так происходит потому что бывают левши, а еще бывают короткие предложения которые показывают одной ркуой левой.
# Типа "следующий слайд" можно левой рукой показать махнуть, даже если ты правша
print(stem_sentences_rsl[0])

[]


In [11]:
#!g1.1
print(gram_sentences_rus[1])

[['ADVPRO'], ['сред', 'несов', 'нп', 'изъяв', 'прош', 'ед', 'V'], ['неод', 'сред', 'S', 'им', 'вин', 'ед'], ['PR'], ['неод', 'S', 'дат', 'пр', 'ед', 'жен'], ['CONJ'], ['неод', 'S', 'мн', 'род', 'жен'], ['сов', 'изъяв', 'прош', 'пе', 'ед', 'муж', 'V'], ['несов', 'инф', 'V', 'пе'], ['неод', 'S', 'мн', 'им', 'вин', 'муж'], ['PR'], ['неод', 'S', 'вин', 'ед', 'гео', 'жен']]


## Замена существительных\глаголов

Надо как-то замену сделать тоже рандомной и прореженной, чтобы не было такого, что одно и то же предложение десять тысяч раз встречается или одни и те же замены.

In [10]:
#!g1.1
len(sentences_rus)

6096

In [14]:
#!g1.1
# вот столько я отложу для более чистого тестирования
0.2 * 6096

1219.2

Можно сделать по от 10 до 30 копий на каждое предложение и это будет прямо большой датасет

Разброс от 10 до 30 нужен, чтобы рандомность какая-то там была. Потому что 6096 это мало предложений, мы не можем говорить, что там сбалансированная выборка по синтаксическим типам предложений, поэтому лучше немножко рандомизировать.

In [11]:
#!g1.1
5800 * 30

174000

- number of sentences to generate from this one k = random.randint(10, 30)
- какие позици в исходном предложении можно заменять: сущ в им.п, вин.п.  (падеж меняю прямо во время генерации, когда уже выбрала одно слово), одуш\неодуш.; паттерны из существительного и полного правого генитива; глаголы переход\непереход.
- Сделать k итераций, на каждой замена всех позиций random.choice(список слов этого типа из словаря(?))
- Сохранить список всех параллельний предложения в tsv вне цикла

## Загрузим списки слов по их свойствам

Из этих списков потом будем выбирать с random.choice

### Сделаем списки существительных

In [209]:
#!g1.1
with open("animate.txt", "r", encoding="utf-8") as f:
    anim_nouns = [line.strip('\n') for line in f.readlines()]

In [210]:
#!g1.1
with open("inanimate.txt", "r", encoding="utf-8") as f:
    inanim_nouns = [line.strip('\n') for line in f.readlines()]

Разделить по роду эти два списка. Должно выйти 6 списков

In [207]:
#!g1.1
def split_by_gender(nouns):
    nouns_m, nouns_f, nouns_n = [], [], []
    for noun in nouns:
        if 'муж' in st.stem(noun)[1][0][0]:
            nouns_m.append(noun)
        elif 'жен' in st.stem(noun)[1][0][0]:
            nouns_f.append(noun)
        elif 'сред' in st.stem(noun)[1][0][0]:
            nouns_n.append(noun)
    return nouns_m, nouns_f, nouns_n

In [216]:
#!g1.1
inan_m_nouns, inan_f_nouns, inan_n_nouns = split_by_gender(inanim_nouns)
anim_m_nouns, anim_f_nouns, anim_n_nouns = split_by_gender(anim_nouns)

In [217]:
#!g1.1
# check len
for el in inan_f_nouns:
    if ' ' in el:
        print(el)

резкая боль
гигиеническая прокладка
горизонтальная плоскость
губная помада
зубная щётка
игральная кость
контурная линия
личная мотивация
мультимедийная презентация
напольная плитка
окружающая среда
подводная лодка
подъемная платформа
спортивная ходьба
устная презентация
чешская республика
шахматная игра
эректильная дисфункция
Слава богу
бутсы футбольные
емкость конденсатора
земля почва
касательная линия
коробка передач
крышка контейнера
лампа индикатор
линия шеи
ось вращения
парная игра
пломба клеймо
подрезка деревьев
проверка контроль
рубка деревьев
ручка дверная
серия визитов
текстура поверхности
угроза здоровью


In [218]:
#!g1.1
anim_f_nouns[:10]

['дочь',
 'племянница',
 'бабушка',
 'внучка',
 'девочка',
 'акула',
 'бабочка',
 'ведьма',
 'вошь',
 'гусеница']

Сюда же я загружу списки с этого гитхаба https://github.com/Koziev/NLP_Datasets 

* Архив patterns.noun_gen.zip содержит паттерны из двух существительных, из которых второе в родительном падеже:

```
4	французские	<<<null>>>
4	дворец	фестивалей
4	названье	мест
4	классы	вагонов
4	доступность	магазина
```

Обратите внимание, что если в исходном предложении у генитива были подчиненные прилагательные или PP, то они в этом датасете будут удалены. Токен `<<>>` в столбце генитива обозначает ситуацию, когда первое существительное употреблено без генитива. Эти записи упрощают маргинализацию частот.

* Архив patterns.noun_np_gen.zip содержит паттерны из существительного и полного правого генитива:

```
окно браузера
течение дня
укус медведки
изюминка такой процедуры
суть декларации
рецепт вкусного молочного коктейля
музыка самого высокого уровня
```


In [220]:
#!g1.1
from string import punctuation

noun_np_gen = []
for line in open("patterns.noun_np_gen.txt", "r", encoding="utf-8"):
    if not (set(line) & set(punctuation)):
        noun_np_gen.append(line.strip('\n'))

In [42]:
#!g1.1
print(len(noun_np_gen))

949109


Разделить на одушевленные\неодушевленные по первому слову:

In [221]:
#!g1.1
noun_np_gen[:10]

['номер телефона',
 'модель кофты',
 'кожа лица кофе',
 'лицо кофе',
 'ребята гор',
 'вид гонок',
 'автор ответа',
 'продажа техники',
 'конец кода',
 'комплекс неполноценности']

In [174]:
#!g1.1
st.stem('висячий замок')

([['висячий', 'замок']],
 [[['A', 'неод', 'ед', 'полн', 'им', 'муж', 'вин'],
   ['неод', 'ед', 'им', 'муж', 'вин', 'S']]])

/kernel/lib/python3.7/site-packages/ml_kernel/kernel.py:848: UserWarning: The following variables cannot be serialized: st
  warnings.warn(message)


In [222]:
#!g1.1
# разделить на одуш\неодуш
anim_noun_np_gen = []
inanim_noun_np_gen = []
for np in noun_np_gen:
    main_word = np.split()[0]
    if 'неод' in st.stem(main_word)[1][0][0]:
        inanim_noun_np_gen.append(np)
    else:
        anim_noun_np_gen.append(np)

In [45]:
#!g1.1
anim_noun_np_gen[:10]

['ребята гор',
 'автор ответа',
 'поставщик товаров',
 'отчим моего мужа',
 'предводитель вражеской армии',
 'детеныш дикой крысы',
 'заведующая детсада',
 'старейшины ресурса',
 'селфи один глаз',
 'королева проекта']

Разделить по роду

In [223]:
#!g1.1
#st = TextStemmer()

inanim_m_noun_np_gen, inanim_f_noun_np_gen, inanim_n_noun_np_gen = [], [], []
for np in inanim_noun_np_gen:
    noun = np.split()[0]
    if 'муж' in st.stem(noun)[1][0][0]:
        inanim_m_noun_np_gen.append(np)
    elif 'жен' in st.stem(noun)[1][0][0]:
        inanim_f_noun_np_gen.append(np)
    elif 'сред' in st.stem(noun)[1][0][0]:
        inanim_n_noun_np_gen.append(np)

In [224]:
#!g1.1
inanim_m_noun_np_gen[:10]

['номер телефона',
 'вид гонок',
 'конец кода',
 'комплекс неполноценности',
 'массаж тела',
 'укус медведки',
 'рецепт вкусного молочного коктейля',
 'спутник жизни',
 'голос певца',
 'скриншот целой страницы']

Тоже самое для одушевленных сочетаний с генитивом

In [225]:
#!g1.1
anim_m_noun_np_gen, anim_f_noun_np_gen, anim_n_noun_np_gen = [], [], []
for np in anim_noun_np_gen:
    noun = np.split()[0]
    if 'муж' in st.stem(noun)[1][0][0]:
        anim_m_noun_np_gen.append(np)
    elif 'жен' in st.stem(noun)[1][0][0]:
        anim_f_noun_np_gen.append(np)
    elif 'сред' in st.stem(noun)[1][0][0]:
        anim_n_noun_np_gen.append(np)

In [226]:
#!g1.1
anim_f_noun_np_gen[:10]

['заведующая детсада',
 'королева проекта',
 'прима скорпионов для бизнеса',
 'уполномочена полиции',
 'дочь отношения',
 'сестра мужа для мамы жены',
 'мама жены',
 'девушки полуночниц',
 'хранительница очага',
 'укротительница тигров']

### Сделаем списки глаголов

Переходные и непереходные. Делаю из жестов, которые есть в нашем словаре.

In [227]:
#!g1.1
words = pd.read_csv("animation_pos.csv", encoding="utf-8")

In [228]:
#!g1.1
words[words['pos'] == 'V']['analysis'][298]

"[{'text': 'торопиться', 'analysis': [{'wt': 1, 'lex': 'торопиться', 'gr': 'V,несов,нп=инф'}]}]"

In [229]:
#!g1.1
# Сколько тут вообще всего глаголов?
len(words[words['pos'] == 'V']['analysis'])

749

Делаю список непереходных глаголов:

In [230]:
#!g1.1
tr_verbs, intr_verbs = [], []
for verb in words[words['pos'] == 'V'].itertuples():
    analysis = json.loads(verb.analysis.replace('\'', '\"'))
    gr = analysis[0]['analysis'][0]['gr']
    if 'пе' in gr:
        tr_verbs.append(verb.text)
    elif 'нп' in gr:
        intr_verbs.append(verb.text)

In [231]:
#!g1.1
random.choice(intr_verbs)

'приостановиться'

In [232]:
#!g1.1
L = [[[3]], [3, 4]]
depth = lambda L: isinstance(L, list) and max(map(depth, L))+1
depth(L)

3

In [58]:
#!g1.1
list(drop_inner_nest(L))

[[3], [3, 4]]

### Генерируем предложения

In [233]:
#!g1.1
depth = lambda L: isinstance(L, list) and max(map(depth, L))+1

def flatten(S):
    if S == []:
        return S
    if isinstance(S[0], list):
        return flatten(S[0]) + flatten(S[1:])
    return S[:1] + flatten(S[1:])

def drop_inner_nest(S):
    for sublist in S:
        if isinstance(sublist[0], list):
            yield sublist[0]
        else:
            yield sublist

def create_gr_n_tags(gr, old_gram_sent_rus):
    # keep original case and number
    tags = ['им', 'вин', 'род', 'дат', 'твор', 'пр', 'парт', 'местн', 'зват', 'ед', 'мн']
    initial_tags = [el for el in old_gram_sent_rus if el in tags]
    for i, word_gr in enumerate(gr):
        gr[i] = [el for el in word_gr if el not in tags]
        gr[i].extend(initial_tags)
    return gr

def create_rsl_gloss(old_rus_lemma, new_rus_lemma, old_stem_sent_rus: list, old_stem_rsl: list):
    """creates rsl glosses for the whole stemmed rsl sentence"""
    # update stemmed rsl sentence
    # return full rsl sentence as a list, not just updated words
    # this does not work well because of things like посмотреть\смотреть
    new_stem_rsl = copy.deepcopy(old_stem_rsl)
    #print('create_rsl_gloss')
    #print('new_rus_lemma', new_rus_lemma)
    found = False
    for i, gloss in enumerate(old_stem_rsl):
        if gloss == old_rus_lemma:
            #print('rus lemma based change')
            new_stem_rsl[i] = new_rus_lemma
            found = True
    if not found:
        # so we have to use index replacement as well
        for j, gloss in enumerate(old_stem_sent_rus):
            if gloss == old_rus_lemma and j < len(new_stem_rsl):
                #print('index based change')
                new_stem_rsl[j] = new_rus_lemma
    #print("new_stem_rsl on this step")
    #pprint(new_stem_rsl)
    return new_stem_rsl

def make_replacments(replacement, old_stem_sent_rus_one, old_gram_sent_rus_one, old_stem_sent_rus: list, old_stem_rsl: list, keep_gr=2):
    old_rus_lemma = old_stem_sent_rus_one
    old_stem_sent_rus_one, gr = st.stem(replacement)
    new_rus_lemma = old_stem_sent_rus_one[0]
    if keep_gr==1:
        old_gram_sent_rus_one = create_gr_n_tags(gr, old_gram_sent_rus_one)
    elif keep_gr==0:
        old_gram_sent_rus_one = gr
    new_stem_rsl_one = create_rsl_gloss(old_rus_lemma, new_rus_lemma, old_stem_sent_rus, old_stem_rsl)
    return old_stem_sent_rus_one, old_gram_sent_rus_one, new_stem_rsl_one

def pick_and_replace(old_stem_sent_rus_one, old_gram_sent_rus_one, old_stem_sent_rus, old_stem_rsl, choices, complex_choices=None):
    if complex_choices:
        # decide between single noun of np with gen
        pick = random.randint(0,1)  # 0 is single noun, 1 is np with gen
        if not bool(pick):
            choice = random.choice(choices)
            if len(choice.split()) > 1:
                old_stem_sent_rus_one, old_gram_sent_rus_one, new_stem_rsl = make_replacments(choice, old_stem_sent_rus_one, old_gram_sent_rus_one, old_stem_sent_rus, old_stem_rsl, keep_gr=1)
                # this line is in the wrong place
            else:
                old_stem_sent_rus_one, old_gram_sent_rus_one, new_stem_rsl = make_replacments(choice, old_stem_sent_rus_one, old_gram_sent_rus_one, old_stem_sent_rus, old_stem_rsl, keep_gr=2)
        else:
            choice = random.choice(complex_choices)
            old_stem_sent_rus_one, old_gram_sent_rus_one, new_stem_rsl = make_replacments(choice, old_stem_sent_rus_one, old_gram_sent_rus_one, old_stem_sent_rus, old_stem_rsl, keep_gr=0)
    else: 
        choice = random.choice(choices)
        old_stem_sent_rus_one, old_gram_sent_rus_one, new_stem_rsl = make_replacments(choice, old_stem_sent_rus_one, old_gram_sent_rus_one, old_stem_sent_rus, old_stem_rsl, keep_gr=0)
    return old_stem_sent_rus_one, old_gram_sent_rus_one, new_stem_rsl

def generate_parallel_sentence(old_gram_sent_rus_init, old_stem_sent_rus_init, old_stem_rsl_init):
    """this function generates one parallel sentence"""
    old_gram_sent_rus = copy.deepcopy(old_gram_sent_rus_init)
    old_stem_sent_rus = copy.deepcopy(old_stem_sent_rus_init)
    old_stem_rsl = copy.deepcopy(old_stem_rsl_init)
    number_replacements = 0   
    for k, gram in enumerate(old_gram_sent_rus_init):
        # gender is relevant for nouns because of concord 
        if 'S' and 'од' and 'муж' in gram:
            number_replacements += 1
            old_stem_sent_rus[k], old_gram_sent_rus[k], old_stem_rsl = pick_and_replace(old_stem_sent_rus[k], old_gram_sent_rus[k], old_stem_sent_rus, old_stem_rsl, anim_m_nouns, complex_choices=anim_m_noun_np_gen)
        elif 'S' and 'неод'  and 'муж' in gram:
            number_replacements += 1 
            old_stem_sent_rus[k], old_gram_sent_rus[k], old_stem_rsl = pick_and_replace(old_stem_sent_rus[k], old_gram_sent_rus[k], old_stem_sent_rus, old_stem_rsl, inan_m_nouns, complex_choices=inanim_m_noun_np_gen)
        elif 'S' and 'од' and 'жен' in gram:
            number_replacements += 1
            old_stem_sent_rus[k], old_gram_sent_rus[k], old_stem_rsl = pick_and_replace(old_stem_sent_rus[k], old_gram_sent_rus[k], old_stem_sent_rus, old_stem_rsl, anim_f_nouns, complex_choices=anim_f_noun_np_gen)
        elif 'S' and 'неод'  and 'жен' in gram:
            number_replacements += 1
            old_stem_sent_rus[k], old_gram_sent_rus[k], old_stem_rsl = pick_and_replace(old_stem_sent_rus[k], old_gram_sent_rus[k], old_stem_sent_rus, old_stem_rsl, inan_f_nouns, complex_choices=inanim_f_noun_np_gen)
        elif 'S' and 'од' and 'сред' in gram:
            number_replacements += 1
            old_stem_sent_rus[k], old_gram_sent_rus[k], old_stem_rsl = pick_and_replace(old_stem_sent_rus[k], old_gram_sent_rus[k], old_stem_sent_rus, old_stem_rsl, anim_n_nouns, complex_choices=anim_n_noun_np_gen)
        elif 'S' and 'неод'  and 'сред' in gram:
            number_replacements += 1
            old_stem_sent_rus[k], old_gram_sent_rus[k], old_stem_rsl = pick_and_replace(old_stem_sent_rus[k], old_gram_sent_rus[k], old_stem_sent_rus, old_stem_rsl, inan_n_nouns, complex_choices=inanim_n_noun_np_gen)
        elif 'V' and 'пе' in gram:
            number_replacements += 1
            old_stem_sent_rus[k], old_gram_sent_rus[k], old_stem_rsl = pick_and_replace(old_stem_sent_rus[k], old_gram_sent_rus[k], old_stem_sent_rus, old_stem_rsl, tr_verbs)
        elif 'V' and 'нп' in gram:
            number_replacements += 1
            old_stem_sent_rus[k], old_gram_sent_rus[k], old_stem_rsl = pick_and_replace(old_stem_sent_rus[k], old_gram_sent_rus[k], old_stem_sent_rus, old_stem_rsl, intr_verbs)

        # если делать больше 5 замен, получается полный бред
        # надо еще сделать замены не подряд, а типа рандомно пропускать шаги??
        # может после переходного глагола всегда пропускать,
        # потому что очень сложно случайным образом подобрать правильный прямой объект или и того хуже инфинитивный оборот
        if number_replacements == 5:
            break
            
    if number_replacements == 0:
        # havent found anything to replace
        return None, None, None
    
    # randomly pick word
    # insert constituent in old_stem_sent_rus (stem\gram before insertion)
    # insert gram of constituent in old_gram_sent_rus it should inherent everything from the original word
    # but if the length of replacement > 1, 
    # insert stemmed constituent in old_stem_rsl
    # flatten lists of depth 3
    if depth(old_gram_sent_rus) > 2:
        old_gram_sent_rus = list(drop_inner_nest(list(drop_inner_nest(old_gram_sent_rus))))

    return old_gram_sent_rus, list(flatten(old_stem_sent_rus)), list(flatten(old_stem_rsl))

In [59]:
#!g1.1
print(len(stem_sentences_rus[0]), len(stem_sentences_rsl[0]))
print(stem_sentences_rus[1], stem_sentences_rsl[2])

2 0
['там', 'быть', 'ограничение', 'по', 'сумма', 'и', 'вога', 'начинать', 'перевозить', 'товар', 'в', 'россия'] []


In [16]:
#!g1.1
stem_sentences_rus[240:242]

[['человек',
  'и',
  'животное',
  'получать',
  'хороший',
  'физический',
  'нагрузка',
  'и',
  'много',
  'положительный',
  'эмоция'],
 ['пожарный',
  'решать',
  'навещать',
  'собака',
  'и',
  'свой',
  'активность',
  'они',
  'показывать',
  'прохожий',
  'что',
  'собака',
  'нуждаться',
  'в',
  'любовь',
  'и',
  'дом']]

### Отделение тестовой выборки

Чтобы быть уверенными, что это прямо реально zero-shot. И есть предложения на основе, которых мы не генерировали обучающую выборку.

In [238]:
#!g1.1
from sklearn.model_selection import train_test_split

stem_sents_rus, test_stem_rus, gram_sents_rus, test_gram_rus, stem_sents_rsl, test_rsl = train_test_split(stem_sentences_rus,
                                                                                                          gram_sentences_rus,
                                                                                                          stem_sentences_rsl,
                                                                                                          test_size=0.2)

In [243]:
#!g1.1
# test test test
from tqdm import tqdm

new_gram_sents_rus, new_stem_sents_rus, new_stem_rsls = [], [], []

for n, gram in enumerate(tqdm(gram_sents_rus)):
    k = random.randint(10, 30)
    for i in range(k): 
        #generate one parallel sentence
        
        if stem_sents_rsl[n] is None or stem_sents_rsl[n] == []:
            break
            
        #print("INITIAL GRAM", gram_sents_rus[n])
        #print("INITIAL RUS", stem_sents_rus[n])
        #print("INITIAL RSL", stem_sents_rsl[n])
        new_gram_sent_rus, new_stem_sent_rus, new_stem_rsl = generate_parallel_sentence(gram_sents_rus[n],
                                                                                        stem_sents_rus[n],
                                                                                        stem_sents_rsl[n])
        
        new_gram_sents_rus.append(new_gram_sent_rus)
        new_stem_sents_rus.append(new_stem_sent_rus)
        new_stem_rsls.append(new_stem_rsl)
        #print("NEW GRAM ", new_gram_sent_rus)
        #print("NEW RUS ", new_stem_sent_rus)
        #print("NEW RSL ", new_stem_rsl)
        #print("\n")

100%|██████████| 4876/4876 [00:50<00:00, 97.41it/s] 


In [244]:
#!g1.1
# save to some tsv file
generated_dict = {'new_stem_sent_rus': new_stem_sents_rus, 'new_gram_sent_rus': new_gram_sents_rus, 'new_stem_rsl': new_stem_rsls}
generated_data = pd.DataFrame.from_dict(generated_dict)

In [245]:
#!g1.1
generated_data.head()

,new_stem_sent_rus,new_gram_sent_rus,new_stem_rsl
0,"[пожалуйста, расставаться, мы, казак, зарубежь...","[[PART], [инф, нп, сов, V], [SPRO, 1-л, мн, да...","[1ps, просить, 2ps, расставаться, 2ps, 1ps, ес..."
1,"[пожалуйста, нападать, мы, чемпион, если, не, вы]","[[PART], [несов, сов, инф, нп, V], [SPRO, 1-л,...","[1ps, просить, 2ps, нападать, 2ps, 1ps, если, ..."
2,"[пожалуйста, сомневаться, мы, хозяин, ресторан...","[[PART], [инф, нп, несов, V], [SPRO, 1-л, мн, ...","[1ps, просить, 2ps, сомневаться, 2ps, 1ps, есл..."
3,"[пожалуйста, оставаться, мы, краб, если, не, вы]","[[PART], [полн, им, вин, несов, неод, ед, дейс...","[1ps, просить, 2ps, оставаться, 2ps, 1ps, если..."
4,"[пожалуйста, жениться, мы, инвалид, если, не, вы]","[[PART], [несов, сов, инф, нп, V], [SPRO, 1-л,...","[1ps, просить, 2ps, жениться, 2ps, 1ps, если, ..."


In [254]:
#!g1.1
new_gram_sents_rus[500]

[['APRO', 'неод', 'вин', 'мн', 'им'],
 ['полн', 'им', 'вин', 'ед', 'сред', 'A'],
 ['им', 'мн', 'муж', 'од', 'S'],
 ['дат', 'твор', 'ед', 'неод', 'S', 'вин', 'мн', 'муж', 'им', 'род', 'пр'],
 ['инф', 'пе', 'несов', 'V'],
 ['PR'],
 ['SPRO', 'род', 'вин']]

In [246]:
#!g1.1
len(generated_data)

59154

In [247]:
#!g1.1
#compression_opts = dict(method='zip', archive_name='generated_stemed.csv') 
#generated_data.to_csv('generated_stemed.zip', index=False, compression=compression_opts)  
generated_data.to_csv('generated_stemed.csv', index=False)  

## Load previously generated data

In [269]:
#!g1.1
generated_data = pd.read_csv('generated_stemed.csv', encoding='utf-8')

In [270]:
#!g1.1
generated_data = generated_data.dropna()

In [271]:
#!g1.1
# test
for row in generated_data.itertuples():
    if isinstance(row[1], float):
        print(row)

In [272]:
#!g1.1
generated_data['new_stem_sent_rus'] = generated_data['new_stem_sent_rus'].apply(lambda sent: sent.strip('[]\'').split('\', \''))
generated_data['new_stem_rsl'] = generated_data['new_stem_rsl'].apply(lambda sent: sent.strip('[]\'').split('\', \''))

#generated_data['new_stem_sent_rus'][0].strip('[]\'').split('\', \'')

In [273]:
#!g1.1
generated_data['new_gram_sent_rus'] = generated_data['new_gram_sent_rus'].apply(lambda sent: json.loads(sent.replace('\'', '\"')))

# Обучаем модель (о да)

### Сначала попробую не прореживать данные и взять все

Можно не прогонять остальные клетки кода, а просто импортировать файл `generated_stemed.csv`

Мне не нужно делать сплит, я могу попробовать учить на всех данных, а тестировать на исходных, типа zero-shot

In [306]:
#!g1.1
from sklearn.model_selection import train_test_split

train_rus, test_rus, train_gram_rus, test_gram_rus, train_rsl, test_rsl = train_test_split(new_stem_sents_rus,
                                                                                           new_gram_sents_rus,
                                                                                           new_stem_rsls,
                                                                                           test_size=0.2)

In [307]:
#!g1.1
voc_rus = VocabularyVectorizer(phrase_border=True)
bin_gram = GramBinarizer(phrase_border=True)
voc_rsl = VocabularyVectorizer(phrase_border=True)

In [308]:
#!g1.1
type(train_rus)

list

In [309]:
#!g1.1
voc_rus = voc_rus.fit(train_rus)
bin_gram = bin_gram.fit(train_gram_rus)
voc_rsl = voc_rsl.fit(train_rsl)

TypeError: object of type 'NoneType' has no len()

In [ ]:
#!g1.1
vec_sentences_rus_train = voc_rus.text_to_index(train_rus)
vec_gram_train = bin_gram.transform(train_gram_rus)
vec_sentences_rsl_train = voc_rsl.text_to_index(train_rsl)

In [86]:
#!g1.1
len(new_stem_sents_rus), voc_rus.word_count, voc_rus.max_len, voc_rsl.word_count, voc_rsl.max_len

(73690, 4965, 50, 3214, 62)

In [88]:
#!g1.1
from collections import Counter

words = [w for s in new_stem_rsls for w in s]

TypeError: 'NoneType' object is not iterable

In [89]:
#!g1.1
Counter(words)['prtcl']

0

## Net

In [248]:
#!g1.1
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn import functional as F

from model.model import Translator

In [250]:
#!g1.1
def set_seed_everywhere(seed, cuda):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed_all(seed)

def handle_dirs(dirpath):
    if not os.path.exists(dirpath):
        os.makedirs(dirpath)

def make_train_state(args):
    return {'stop_early': False,
            'early_stopping_step': 0,
            'early_stopping_best_val': 1e8,
            'learning_rate': args.learning_rate,
            'epoch_index': 0,
            'train_loss': [],
            'train_acc': [],
            'val_loss': [],
            'val_acc': [],
            'test_loss': -1,
            'test_acc': -1,
            'model_filename': args.model_state_file}

def update_train_state(args, model, train_state):
    """Handle the training state updates.
    Components:
     - Early Stopping: Prevent overfitting.
     - Model Checkpoint: Model is saved if the model is better
    
    :param args: main arguments
    :param model: model to train
    :param train_state: a dictionary representing the training state values
    :returns:
        a new train_state
    """

    # Save one model at least
    if train_state['epoch_index'] == 0:
        torch.save(model.state_dict(), train_state['model_filename'])
        train_state['stop_early'] = False

    # Save model if performance improved
    elif train_state['epoch_index'] >= 1:
        loss_tm1, loss_t = train_state['val_loss'][-2:]
         
        # If loss worsened
        if loss_t >= loss_tm1:
            # Update step
            train_state['early_stopping_step'] += 1
        # Loss decreased
        else:
            # Save the best model
            if loss_t < train_state['early_stopping_best_val']:
                torch.save(model.state_dict(), train_state['model_filename'])
                train_state['early_stopping_best_val'] = loss_t

            # Reset early stopping step
            train_state['early_stopping_step'] = 0

        # Stop early ?
        train_state['stop_early'] = \
            train_state['early_stopping_step'] >= args.early_stopping_criteria

    return train_state

def normalize_sizes(y_pred, y_true):
    """Normalize tensor sizes
    
    Args:
        y_pred (torch.Tensor): the output of the model
            If a 3-dimensional tensor, reshapes to a matrix
        y_true (torch.Tensor): the target predictions
            If a matrix, reshapes to be a vector
    """
    if len(y_pred.size()) == 3:
        y_pred = y_pred.contiguous().view(-1, y_pred.size(2))
    if len(y_true.size()) == 2:
        y_true = y_true.contiguous().view(-1)
    return y_pred, y_true

def compute_accuracy(y_pred, y_true, mask_index):
    y_pred, y_true = normalize_sizes(y_pred, y_true)

    _, y_pred_indices = y_pred.max(dim=1)
    
    correct_indices = torch.eq(y_pred_indices, y_true).float()
    valid_indices = torch.ne(y_true, mask_index).float()
    
    n_correct = (correct_indices * valid_indices).sum().item()
    n_valid = valid_indices.sum().item()

    return n_correct / n_valid * 100

def sequence_loss(y_pred, y_true, mask_index):
    y_pred, y_true = normalize_sizes(y_pred, y_true)
    return F.cross_entropy(y_pred, y_true, ignore_index=mask_index)

In [251]:
#!g1.1
import random 

def batch_generator(rus_data, rsl_data, batch_size=32):
    rus_data = np.array(rus_data)
    rsl_data = np.array(rsl_data)
    
    data_length = len(rus_data)
    tail_length = batch_size - data_length % batch_size
    index = list(range(data_length))
    random.shuffle(index)
    
    index = np.array(index + random.choices(index, k=tail_length))
    num_batches = len(index) // batch_size
    index = index.reshape((num_batches, batch_size))
    
    for batch_ind, inds in enumerate(tqdm(index)):
        yield batch_ind, torch.tensor(rus_data[inds]), torch.tensor(rsl_data[inds])

In [93]:
#!g1.1
from argparse import Namespace

args = Namespace(
    seed=1337,
    learning_rate=5e-4,
    batch_size=64,
    num_epochs=30,  # 30
    rus_emb_size=16,
    rsl_emb_size = 16,
    rnn_size = 64,
    early_stopping_criteria=5,
    mask_index = voc_rsl.mask_ind
)

set_seed_everywhere(args.seed, torch.cuda.is_available())

model = Translator(voc_rus.word_count, args.rus_emb_size, voc_rsl.word_count, args.rsl_emb_size, args.rnn_size, voc_rsl.bos_ind)

optimizer = optim.Adam(model.parameters(), lr=args.learning_rate)

In [94]:
#!g1.1

for epoch_index in range(args.num_epochs):
    # sample_probability = (10 + epoch_index) / args.num_epochs
    if epoch_index < 0.5 * args.num_epochs:
        sample_probability = 0.05
    else:
        sample_probability = ( 2 * (epoch_index+1) - args.num_epochs) / args.num_epochs
    
    running_loss = 0.0
    running_acc = 0.0
    model.train()
    
    for batch_ind, rus_batch, rsl_batch in batch_generator(vec_sentences_rus_train, vec_sentences_rsl_train, args.batch_size):
        optimizer.zero_grad()
        
        y_pred = model(rus_batch, rsl_batch, 0.0)
        
        loss = sequence_loss(y_pred, rsl_batch, args.mask_index)
        
        loss.backward()
        
        optimizer.step()
        
        running_loss += (loss.item() - running_loss) / (batch_ind + 1)
        acc_t = compute_accuracy(y_pred, rsl_batch, args.mask_index)
        
    print('Epoch: ', epoch_index, 'Loss: ', running_loss, '\tacc: ', acc_t, '\tsample_prob: ', sample_probability)

100%|██████████| 77/77 [00:28<00:00,  2.68it/s]


Epoch:  0 Loss:  6.1725486903995685 	acc:  27.586206896551722 	sample_prob:  0.05
Epoch:  1 Loss:  5.013691728765313 	acc:  26.778242677824267 	sample_prob:  0.05
Epoch:  2 Loss:  4.8191376599398525 	acc:  30.416666666666664 	sample_prob:  0.05
Epoch:  3 Loss:  4.484796570493028 	acc:  36.98979591836735 	sample_prob:  0.05
Epoch:  4 Loss:  4.246925604807866 	acc:  34.146341463414636 	sample_prob:  0.05
Epoch:  5 Loss:  3.952063675050611 	acc:  40.73226544622426 	sample_prob:  0.05
Epoch:  6 Loss:  3.6457010863663317 	acc:  41.60839160839161 	sample_prob:  0.05
Epoch:  7 Loss:  3.3322053543933023 	acc:  47.90940766550523 	sample_prob:  0.05
Epoch:  8 Loss:  3.0339383212002833 	acc:  58.536585365853654 	sample_prob:  0.05
Epoch:  9 Loss:  2.7445455619267047 	acc:  58.35189309576837 	sample_prob:  0.05
Epoch:  10 Loss:  2.490479747970383 	acc:  62.21294363256785 	sample_prob:  0.05
Epoch:  11 Loss:  2.250601181736241 	acc:  67.83625730994152 	sample_prob:  0.05
Epoch:  12 Loss:  2.0226153

In [95]:
#!g1.1
model = model.eval()

In [96]:
#!g1.1
regex = re.compile(r'(?:<bos>|<eos>.*)')
def pred_vs_rsl(tensor, true_rsl, rus):
    
    tensor = tensor.argmax(2).tolist()
    
    pred_rsl = voc_rsl.index_to_text(tensor)
    true_rsl = voc_rsl.index_to_text(true_rsl)
    rus = voc_rus.index_to_text(rus)
                   
    f = lambda x: regex.sub('', " ".join(x))
                   
    pred_rsl = [f(sentence) for sentence in pred_rsl]
    true_rsl = [f(sentence) for sentence in true_rsl]
    true_rus = [f(sentence) for sentence in rus]
    
    return pred_rsl, true_rsl, true_rus

### Тестируем модель 

In [97]:
#!g1.1
vec_sentences_rus_test = voc_rus.text_to_index(test_rus)
vec_gram_test = bin_gram.transform(test_gram_rus)
vec_sentences_rsl_test = voc_rsl.text_to_index(test_rsl)

In [98]:
#!g1.1
y_pred = model(torch.tensor(vec_sentences_rus_test), torch.tensor(vec_sentences_rsl_test), 0)

In [99]:
#!g1.1
trans, truth_rsl, rus = pred_vs_rsl(y_pred,
                                    torch.tensor(vec_sentences_rsl_test),
                                    torch.tensor(vec_sentences_rus_test))

In [100]:
#!g1.1
# lets have a look
for i in range(10):
    print("RSL: ", truth_rsl[i])
    print("TRANS: ", trans[i])
    print("RUS: ", rus[i])
    print('\n')

RSL:   но быть руководитель простой не.хотеть принимать 
TRANS:   но быть руководитель простой не.хотеть принимать 
RUS:   но бывать руководитель просто не хотеть принимать 


RSL:   
TRANS:   
RUS:   в общий я много где выступать в разный город 


RSL:   
TRANS:   
RUS:   у я нет машина 


RSL:   обращение 2ps крепкий <unk> и прощаться 
TRANS:   старый 2ps крепкий радостный и прощаться 
RUS:   <unk> крепко и <unk> 


RSL:   следующий2 мировоззрение восприятие думать мнение pl о мир2 у.всех.одинаково 
TRANS:   следующий2 вспоминать можно думать мнение pl о мир2 у.всех.одинаково 
RUS:   следующий одинаковый мировоззрение и восприятие мир 


RSL:   1ps видеть не мочь человек молодой н-о 3ps <unk> 1ps о clf два.человека <dact> мочь 
TRANS:   1ps видеть не мочь человек молодой н-о 3ps необходимость 1ps о clf к <dact> мочь 
RUS:   я не мочь принимать это потому что он быть молодой но он заботиться о мы с дочка как мочь 


RSL:   
TRANS:   
RUS:   я идти гриб собирать и смотреть парень военн

### Теперь считаем bleu-score

In [101]:
#!g1.1
len(trans), len(truth_rsl)

(1220, 1220)

In [102]:
#!g1.1
import sacrebleu

rus_rsl_bleu = sacrebleu.corpus_bleu(trans, [truth_rsl])
print("--------------------------")
print("Russian to RSL: ", rus_rsl_bleu.score)

--------------------------
Russian to RSL:  72.08847139943303


# Теперь zero-shot

In [310]:
#!g1.1
voc_rus = VocabularyVectorizer(phrase_border=True)
bin_gram = GramBinarizer(phrase_border=True)
voc_rsl = VocabularyVectorizer(phrase_border=True)

In [311]:
#!g1.1
voc_rus = voc_rus.fit(list(generated_data['new_stem_sent_rus']))  # list(generated_data['new_stem_sent_rus']
bin_gram = bin_gram.fit(list(generated_data['new_gram_sent_rus']))
voc_rsl = voc_rsl.fit(list(generated_data['new_stem_rsl']))

In [312]:
#!g1.1
vec_sentences_rus_train = voc_rus.text_to_index(list(generated_data['new_stem_sent_rus']))
vec_gram_train = bin_gram.transform(list(generated_data['new_gram_sent_rus']))
vec_sentences_rsl_train = voc_rsl.text_to_index(list(generated_data['new_stem_rsl']))

In [278]:
#!g1.1
len(new_stem_sents_rus), len(generated_data['new_stem_sent_rus']), voc_rus.word_count, voc_rus.max_len, voc_rsl.word_count, voc_rsl.max_len

(59154, 57604, 14763, 61, 14707, 62)

In [279]:
#!g1.1
words = [w for s in list(generated_data['new_stem_rsl']) for w in s]

### Обучаем модель на всех-всех сгенерированных данных

плюс тут удалены пустые строки ржя

In [313]:
#!g1.1
from argparse import Namespace

args = Namespace(
    seed = 1337,
    learning_rate = 5e-4, 
    batch_size = 96,  # 64
    num_epochs = 10,  # 30
    rus_emb_size = 16,  # 16
    rsl_emb_size = 16,  # 16
    rnn_size = 64,  # 64
    early_stopping_criteria = 5,
    mask_index = voc_rsl.mask_ind
)

set_seed_everywhere(args.seed, torch.cuda.is_available())

model_zero = Translator(voc_rus.word_count, args.rus_emb_size, voc_rsl.word_count, args.rsl_emb_size, args.rnn_size, voc_rsl.bos_ind)

optimizer = optim.Adam(model_zero.parameters(), lr=args.learning_rate)

In [314]:
#!g1.1

for epoch_index in range(args.num_epochs):
    # sample_probability = (10 + epoch_index) / args.num_epochs
    if epoch_index < 0.5 * args.num_epochs:
        sample_probability = 0.05
    else:
        sample_probability = ( 2 * (epoch_index+1) - args.num_epochs) / args.num_epochs
    
    running_loss = 0.0
    running_acc = 0.0
    model_zero.train()
    
    for batch_ind, rus_batch, rsl_batch in batch_generator(vec_sentences_rus_train, vec_sentences_rsl_train, args.batch_size):
        optimizer.zero_grad()
        
        y_pred = model_zero(rus_batch, rsl_batch, 0.0)
        
        loss = sequence_loss(y_pred, rsl_batch, args.mask_index)
        
        loss.backward()
        
        optimizer.step()
        
        running_loss += (loss.item() - running_loss) / (batch_ind + 1)
        acc_t = compute_accuracy(y_pred, rsl_batch, args.mask_index)
        
    print('Epoch: ', epoch_index, 'Loss: ', running_loss, '\tacc: ', acc_t, '\tsample_prob: ', sample_probability)

100%|██████████| 601/601 [14:27<00:00,  1.44s/it]


Epoch:  0 Loss:  6.045795564445204 	acc:  32.06349206349206 	sample_prob:  0.05
Epoch:  1 Loss:  3.7826472832240166 	acc:  61.92893401015228 	sample_prob:  0.05
Epoch:  2 Loss:  1.936426304937799 	acc:  84.00918133129304 	sample_prob:  0.05
Epoch:  3 Loss:  0.9579724243000619 	acc:  92.5754060324826 	sample_prob:  0.05
Epoch:  4 Loss:  0.5302181895481369 	acc:  95.78220858895705 	sample_prob:  0.05
Epoch:  5 Loss:  0.33177723672346326 	acc:  97.74669774669775 	sample_prob:  0.2
Epoch:  6 Loss:  0.22643735915571223 	acc:  97.9182156133829 	sample_prob:  0.4
Epoch:  7 Loss:  0.1629948131306198 	acc:  98.27067669172932 	sample_prob:  0.6
Epoch:  8 Loss:  0.11937540788172493 	acc:  99.47526236881559 	sample_prob:  0.8
Epoch:  9 Loss:  0.09096824365998057 	acc:  99.04686258935664 	sample_prob:  1.0


In [315]:
#!g1.1
model_zero = model_zero.eval()

In [316]:
#!g1.1
regex = re.compile(r'(?:<bos>|<eos>.*)')
def pred_vs_rsl(tensor, true_rsl, rus):
    
    tensor = tensor.argmax(2).tolist()
    
    pred_rsl = voc_rsl.index_to_text(tensor)
    true_rsl = voc_rsl.index_to_text(true_rsl)
    rus = voc_rus.index_to_text(rus)
                   
    f = lambda x: regex.sub('', " ".join(x))
                   
    pred_rsl = [f(sentence) for sentence in pred_rsl]
    true_rsl = [f(sentence) for sentence in true_rsl]
    true_rus = [f(sentence) for sentence in rus]
    
    return pred_rsl, true_rsl, true_rus

### Тестируем модель zero-shot

на исходных данных

In [301]:
#!g1.1
for k, phrase in enum(test_gram_rus):
    if phrase

In [303]:
#!g1.1
test_rsl_clean = [phrase for phrase in test_rsl if phrase is not None]

In [298]:
#!g1.1
for phrase in test_gram_rus:
    if phrase is None:
        print(phrase)

In [304]:
#!g1.1
vec_sentences_rus_test = voc_rus.text_to_index(test_stem_rus)
vec_gram_test = bin_gram.transform(test_gram_rus_clean)
vec_sentences_rsl_test = voc_rsl.text_to_index(test_rsl_clean)

In [305]:
#!g1.1
y_pred = model_zero(torch.tensor(vec_sentences_rus_test), torch.tensor(vec_sentences_rsl_test), 0)

RuntimeError: torch.cat(): Sizes of tensors must match except in dimension 1. Got 11532 and 1220 in dimension 0 (The offending index is 1)

In [43]:
#!g1.1
trans, truth_rsl, rus = pred_vs_rsl(y_pred,
                                    torch.tensor(vec_sentences_rsl_test),
                                    torch.tensor(vec_sentences_rus_test))

In [44]:
#!g1.1
# lets have a look
for i in range(10):
    print("RSL: ", truth_rsl[i])
    print("TRANS: ", trans[i])
    print("RUS: ", rus[i])
    print('\n')

RSL:   indx <nums> есть маленький 3ps эмоция готовый <dact> clf шуба.выбивать сапоги clf обувь.чистить санки готовый порядок эмоция скоро новый год 
TRANS:   indx <nums> есть маленький 3ps эмоция готовый <dact> clf шуба.выбивать сапоги clf обувь.чистить санки готовый порядок эмоция скоро новый год 
RUS:   там быть один маленький дед мороз который очень ждать новый год у он готовый вс рано остальной шуба выбивать сапог начищать и сани готовый 


RSL:   все дед.мороз неинтересно пусть думать что подарок дети <nums> 3ps маленький готово подарок коробка pl готово цвет свой pl 
TRANS:   все дед.мороз неинтересно пусть думать что подарок дети <nums> 3ps маленький готово подарок аппетит pl готово цвет свой pl 
RUS:   пока другой дед мороз быть неинтересно что можно приносить ребенок подарок маленький дед мороз уже давно лежать готовый завертывать в разноцветный бумага 


RSL:   это 3ps идеальный печь печенье <dact> вкус любой это 3ps уметь форма звезда ель разный pl 
TRANS:   это 3ps идеальны

In [46]:
#!g1.1
import sacrebleu

rus_rsl_bleu = sacrebleu.corpus_bleu(trans, [truth_rsl])
print("--------------------------")
print("Russian to RSL: ", rus_rsl_bleu.score)

--------------------------
Russian to RSL:  91.65096920794223
